In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim

import sys

sys.path.append("F:/기계학습특론/saint-main/models")  # SAINT 경로 추가

from model import TabAttention  # TabAttention 클래스 임포트


# 데이터 로드 및 전처리
file_path = "adult.csv"
data = pd.read_csv(file_path)

# '?' 처리 및 결측치 제거
data.replace('?', pd.NA, inplace=True)
data.dropna(inplace=True)

# 범주형 데이터와 연속형 데이터 구분
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 
                       'relationship', 'race', 'gender', 'native-country']
continuous_columns = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# 범주형 데이터 인코딩
label_encoders = {col: LabelEncoder().fit(data[col]) for col in categorical_columns}
for col, encoder in label_encoders.items():
    data[col] = encoder.transform(data[col])

# 타겟 데이터 인코딩
data['income'] = (data['income'] == '>50K').astype(int)

# 입력 데이터 분리
X = data.drop("income", axis=1)
y = data["income"]

# 범주형 및 연속형 데이터 분리
X_categ = X[categorical_columns].values
X_cont = X[continuous_columns].values

# 데이터 분할
X_categ_train, X_categ_test, X_cont_train, X_cont_test, y_train, y_test = train_test_split(
    X_categ, X_cont, y, test_size=0.2, random_state=42
)

# 연속형 데이터 정규화
scaler = StandardScaler()
X_cont_train = scaler.fit_transform(X_cont_train)
X_cont_test = scaler.transform(X_cont_test)

# PyTorch 텐서로 변환
X_categ_train_tensor = torch.tensor(X_categ_train, dtype=torch.long)
X_cont_train_tensor = torch.tensor(X_cont_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_categ_test_tensor = torch.tensor(X_categ_test, dtype=torch.long)
X_cont_test_tensor = torch.tensor(X_cont_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# TabAttention 모델 초기화
model = TabAttention(
    categories=[len(data[col].unique()) for col in categorical_columns],
    num_continuous=len(continuous_columns),
    dim=32,  # 임베딩 차원
    depth=6,  # Transformer 레이어 수
    heads=8,  # Multi-head Attention 수
    dim_head=16,
    dim_out=1,  # 출력 크기
    mlp_hidden_mults=(4, 2),
    mlp_act=nn.ReLU(),
    attentiontype="col"
)

# CPU 설정
device = torch.device("cpu")
model.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 모델 학습
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # 모델 출력
    outputs = model(X_categ_train_tensor, X_cont_train_tensor).squeeze()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# 모델 평가
model.eval()
with torch.no_grad():
    predictions = model(X_categ_test_tensor, X_cont_test_tensor).squeeze()
    predictions = torch.sigmoid(predictions).cpu().numpy()
    pred_labels = (predictions > 0.5).astype(int)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, pred_labels)
    auroc = roc_auc_score(y_test, predictions)
    precision = precision_score(y_test, pred_labels)
    recall = recall_score(y_test, pred_labels)
    f1 = f1_score(y_test, pred_labels)

# 평가 결과 출력
print("\n모델 평가 결과:")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"AUROC: {auroc:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1 점수 (F1-Score): {f1:.4f}")


RuntimeError: The size of tensor a (32) must match the size of tensor b (192) at non-singleton dimension 2